## AutoGIS exercise 5

In [ ]:
# importing necessary modules
import requests
import geojson
from pyproj import CRS
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

In [ ]:
# Read population grid data for 2018 into a variable `pop`. 
# Specify the url for web feature service
url = 'https://kartta.hsy.fi/geoserver/wfs'

# Specify parameters (read data in json format). 
# Available feature types in this particular data source: http://geo.stat.fi/geoserver/vaestoruutu/wfs?service=wfs&version=2.0.0&request=describeFeatureType
params = dict(service='WFS', 
              version='2.0.0', 
              request='GetFeature', 
              typeName='asuminen_ja_maankaytto:Vaestotietoruudukko_2018', 
              outputFormat='json')

# Fetch data from WFS using requests
r = requests.get(url, params=params)

# Create GeoDataFrame from geojson
pop = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

In [ ]:
# Defining the crs
pop.crs = CRS.from_epsg(3879)

In [ ]:
# Importing more data
# Filepaths
roads_fp = "data/roads.shp"
metro_fp = "data/metro.shp"

# Read files
roads = gpd.read_file(roads_fp)
metro = gpd.read_file(metro_fp)

In [ ]:
# Reprojecting geometries for later use with basemap
roads = roads.to_crs(crs=3857)
metro = metro.to_crs(crs=3857)
pop = pop.to_crs(crs=3857)

In [ ]:
# Check CRS names
print("Roads crs:", CRS(roads.crs).name)
print("Metro crs:", CRS(metro.crs).name)
print("Pop crs: ",CRS(pop.crs).name)

In [ ]:
# Defining median age category for each population grid
pop['median_ika'] = ''
for i in range(len(pop)):
    ika = []
    for k in range(4,13,1):
        ika += pop.iloc[i, k] * [pop.columns[k]]
    pop.loc[i, 'median_ika'] = ika[int(len(ika)/2)]

In [ ]:
# Create one subplot. Control figure size in here.
fig, ax = plt.subplots(figsize=(12,8))

# Visualize the travel times into 9 classes using "Quantiles" classification scheme
pop.plot(ax=ax, column='median_ika', cmap='PiYG', linewidth=0.03, legend = True, alpha=0.7)

# Add roads on top of the grid
# (use ax parameter to define the map on top of which the second items are plotted)
roads.plot(ax=ax, color="yellow", linewidth=1.5)

# Add metro on top of the previous map
metro.plot(ax=ax, color="green", linewidth=2.5)

# Remove the empty white-space around the axes
plt.tight_layout()

ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

# Save the figure as png file with resolution of 300 dpi
#outfp = "static_map.png"
#plt.savefig(outfp, dpi=300)